# Debut

In [10]:
import notebook_importer

In [11]:
from Velo        import Velo
from Ville       import Ville
from Trajet      import Trajet
from Logger      import Logger
from Station     import Station
from Cycliste    import Cycliste
from Position    import Position as Position
from Prestataire import Prestataire

import pandas as pd
import importlib

In [12]:
%matplotlib inline

In [13]:
def draw(positions):
    import pandas as pd
    df = pd.DataFrame([ {"lon" : x.longitude, 
                "lat": x.lattitude} for x in positions])
    df.plot.scatter(x="lon", y="lat")
    
    

In [14]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%autoreload 2

In [80]:
import time
durees_totales = []
nb_cycliste    = 20
width          = 10
length         = 10
JcDecaux       = Prestataire(width,length)
paris          = Ville(nb_cycliste, JcDecaux)
positions      = pd.DataFrame()
ax             = None
durees         = []
debut          = time.time()
print(JcDecaux)
for i in range(200):
    local_debut = time.time()
    paris.reveil()
    durees.append(time.time() - local_debut)
total = time.time()-debut
durees_totales.append(total)
print(total)


                Nb de stations           : 100 
                Nb de vélos              : 1325 
                Nb de stations avec vélo : 100 
                Nb de stations pleines   : 0
                Nb de stations vides     : 0
        
0.10091590881347656


In [81]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6, inferno
output_notebook()


Loading BokehJS ...

In [82]:
unique_id_cycliste = list(set([x.id for x in paris.cyclistes]))
unique_id_cycliste = [str(x) for x in unique_id_cycliste]

In [83]:
palette = inferno(len(unique_id_cycliste))

In [84]:
color_map = factor_cmap('id_cycliste',palette = palette, factors =  unique_id_cycliste)

In [85]:
positions             = paris.get_historique_positions_cycliste()[['longitude', 'lattitude', 'id_cycliste', 'time',  ]]
positions.id_cycliste = positions.id_cycliste.map(str)
positions["type"]     = "position"
positions             = ColumnDataSource(positions)


In [86]:


homes                 = paris.get_cycliste_homes()
homes["type"]         = "homes"
homes                 = ColumnDataSource(homes)

workplaces            = paris.get_cycliste_workplaces()
workplaces["type"]    = "workplaces"
workplaces            = ColumnDataSource(workplaces)


stations            = paris.get_topology()
stations["type"]    = "stations"
stations            = ColumnDataSource(stations)

# Create a ColumnDataSource from df: source



# Create the figure: p
p = figure()
p.diamond( 'longitudes', 'lattitude', source= workplaces , color="cyan", size=10)
p.asterisk('longitudes', 'lattitude', source= homes      , color="red" , size=10)
p.square(  'longitudes', 'lattitude', source= stations   , color="pink", size=8)


p.circle('longitude', 'lattitude', source= positions, fill_color=color_map)# Add circle glyphs to the figure p


hover = HoverTool(tooltips=[("cyliste", "@id_cycliste"),
                            ("type"   , "@type")])
p.add_tools(hover)
show(p)
